In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
import time
import glob
import sys
import cv2
import os

import torch

from cellpose import models, core, io, transforms
from cellpose.models import MODEL_NAMES

sys.path.insert(0, "../torchvf/")
from metrics import *

In [2]:
worm_dir       = "../data/bpcis/worm_test/"
bact_fluor_dir = "../data/bpcis/bact_fluor_test/"
bact_phase_dir = "../data/bpcis/bact_phase_test/"

In [3]:
worm_image_files = sorted(glob.glob(os.path.join(worm_dir, "*_img.tif")))
worm_masks_files = sorted(glob.glob(os.path.join(worm_dir, "*_masks.tif")))

bact_fluor_image_files = sorted(glob.glob(os.path.join(bact_fluor_dir, "*_img.tif")))
bact_fluor_masks_files = sorted(glob.glob(os.path.join(bact_fluor_dir, "*_masks.tif")))

bact_phase_image_files = sorted(glob.glob(os.path.join(bact_phase_dir, "*_img.tif")))
bact_phase_masks_files = sorted(glob.glob(os.path.join(bact_phase_dir, "*_masks.tif")))

In [4]:
worm_imgs  = [io.imread(f) for f in worm_image_files]
worm_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in worm_masks_files]

bact_fluor_imgs  = [io.imread(f) for f in bact_fluor_image_files]
bact_fluor_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in bact_fluor_masks_files]

bact_phase_imgs  = [io.imread(f) for f in bact_phase_image_files]
bact_phase_masks = [cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in bact_phase_masks_files]

In [5]:
print(len(bact_phase_imgs), len(bact_phase_masks))
print(len(bact_fluor_imgs), len(bact_fluor_masks))
print(len(worm_imgs), len(worm_masks))

148 148
75 75
60 60


In [6]:
# During evaluation of the H1 and H2 models I didn't evaluate on certain images that 
# I believe have problems with ground truth masks. This makes sure I'm not evaluating 
# Cellpose or Omnipose on these too. These include images 3 and 5 from the bacterial 
# fluorescence subset and image 19 from the worm subset. 

def remove_indices(imgs, masks, indices):
    imgs  = [i for j, i in enumerate(imgs)  if j not in indices]
    masks = [i for j, i in enumerate(masks) if j not in indices]
    
    return imgs, masks

bact_fluor_imgs, bact_fluor_masks = remove_indices(bact_fluor_imgs, bact_fluor_masks, [3, 5])
worm_imgs, worm_masks = remove_indices(worm_imgs, worm_masks, [19])

In [7]:
print(len(bact_phase_imgs), len(bact_phase_masks))
print(len(bact_fluor_imgs), len(bact_fluor_masks))
print(len(worm_imgs), len(worm_masks))

148 148
73 73
59 59


In [8]:
print(MODEL_NAMES)

['cyto', 'nuclei', 'cyto2', 'bact_phase_cp', 'bact_fluor_cp', 'plant_cp', 'worm_cp', 'cyto2_omni', 'bact_phase_omni', 'bact_fluor_omni', 'plant_omni', 'worm_omni', 'worm_bact_omni', 'worm_high_res_omni']


In [9]:
def eval_model(model, images, gt_masks):
    start = time.time()
    
    masks, flows, styles = model.eval(
        images, 
        channels = [0,0],
        rescale = None,
        mask_threshold = -1,
        flow_threshold = 0,
        omni = False,
        resample = True,
        tile = False, 
        transparency = True,
        net_avg = False
    )
    
    out = map_iou(masks, gt_masks)

    f1s = []
    ious = []
    for pred_mask, true_mask in zip(masks, gt_masks):
        pred_mask = torch.Tensor(pred_mask[None][None] > 0)
        true_mask = torch.Tensor(true_mask[None][None] > 0)

        f1s.append(f1(pred_mask, true_mask).numpy())
        ious.append(iou(pred_mask, true_mask).numpy())

    print("F1 Score: ",  np.mean(f1s))
    print("IoU Score: ", np.mean(ious))
    
    print(f"AP_50: {out[0]:.3f} | AP_75: {out[1]:.3f} | AP_90: {out[2]:.3f} | AP: {np.mean(out):.3f}")

In [10]:
model = models.CellposeModel(
    gpu = True, 
    model_type = "worm_cp",
    net_avg = False
)

eval_model(model, worm_imgs, worm_masks)

2022-08-27 13:29:04,452 [INFO] >>worm_cp<< model set to be used
2022-08-27 13:29:05,746 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:29:05,746 [INFO] >>>> using GPU
2022-08-27 13:29:12,270 [INFO] 100%|##########| 59/59 [00:06<00:00,  9.39it/s]
F1 Score:  0.9397965
IoU Score:  0.88704914
AP_50: 0.868 | AP_75: 0.700 | AP_90: 0.292 | AP: 0.620


In [11]:
model = models.CellposeModel(
    gpu = True, 
    model_type = "bact_phase_cp"
)

eval_model(model, bact_phase_imgs, bact_phase_masks)

2022-08-27 13:29:12,737 [INFO] >>bact_phase_cp<< model set to be used
2022-08-27 13:29:12,738 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:29:12,738 [INFO] >>>> using GPU
2022-08-27 13:30:13,526 [INFO] 100%|##########| 148/148 [01:00<00:00,  2.45it/s]
F1 Score:  0.93953717
IoU Score:  0.9037077
AP_50: 0.554 | AP_75: 0.509 | AP_90: 0.345 | AP: 0.469


In [12]:
model = models.CellposeModel(
    gpu = True, 
    model_type = "bact_fluor_cp"
)

eval_model(model, bact_fluor_imgs, bact_fluor_masks)

2022-08-27 13:30:19,170 [INFO] >>bact_fluor_cp<< model set to be used
2022-08-27 13:30:19,172 [INFO] ** TORCH CUDA version installed and working. **
2022-08-27 13:30:19,173 [INFO] >>>> using GPU
2022-08-27 13:30:38,714 [INFO] 100%|##########| 73/73 [00:19<00:00,  3.78it/s]
F1 Score:  0.9751827
IoU Score:  0.951724
AP_50: 0.913 | AP_75: 0.745 | AP_90: 0.056 | AP: 0.571


In [13]:
from cellpose.resnet_torch import CPnet

In [14]:
model = CPnet([1, 32, 64, 128, 256], 1, 3)

In [15]:
param_count = sum(p.numel() for p in model.parameters())

print(f"Model Parameter Count: {param_count}")

Model Parameter Count: 6600453
